In [1]:
import numpy as np
import yaml
from onsager import OnsagerCalc
from onsager import crystal
from onsager import crystalStars as stars
from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
def build_point_group_ops(lattice, basis, threshold=1e-8):

        pgroup_ops = []
        sgroup_ops = []
        inv_lat_vec = np.linalg.inv(lattice)
        supercellvect = [np.array((n0, n1, n2))
                         for n0 in range(-1, 2)
                         for n1 in range(-1, 2)
                         for n2 in range(-1, 2)
                         if (n0, n1, n2) != (0, 0, 0)]
        nmat_list = [X for X in [np.array((n0, n1, n2))
                                 for n0 in supercellvect
                                 for n1 in supercellvect
                                 for n2 in supercellvect] if abs(np.linalg.det(X)) == 1]
        for nmat in nmat_list:
            g = np.dot(lattice, np.dot(nmat, inv_lat_vec))
            if np.all(abs(np.dot(g.T, g) - np.eye(3)) < threshold):
                flag_op = 1
                for bas in basis:
                    vec1 = bas - basis[0]
                    for i, j in enumerate(vec1):
                        if j < 0:
                            vec1[i] += 1.0
                    vec2 = np.dot(nmat, vec1)
                    for i, j in enumerate(vec2):
                        if j < 0:
                            vec2[i] += 1.0
                    if np.any(abs(vec1 - vec2) > threshold):
                        flag_op = 0
                        sgroup_ops.append(nmat)

                if flag_op:
                    pgroup_ops.append(nmat)
        return np.array(pgroup_ops), np.array(sgroup_ops) 
        

def apply_pg_site_dir(pg, site1, site2, threshold=1e-8):
        
    if np.any([np.all(abs(site1 - np.dot(g, site2)) < threshold) for g in pg]):
            return 1
    else:
            return 0

        
def apply_sg_site_dir(sg, site1, site2, threshold=1e-8):
        
    if np.any([np.all(abs(site1 - np.dot(g, site2)) < threshold) for g in sg]):
            return 1
    else:
            return 0

def apply_pg_trans(pg, trans1, trans2, threshold=1e-8):

    if np.any([np.all(abs(trans1[0] - np.dot(g, trans2[0])) < threshold) and np.all(abs(trans1[1] - np.dot(g, trans2[1])) < threshold) for g in pg]):
            return 1
    else:
            return 0

        
def apply_sg_trans(sg, trans1, trans2, threshold=1e-8):
        
    if np.any([np.all(abs(trans1[0] - np.dot(g, trans2[0])) < threshold)and np.all(abs(trans1[1] - np.dot(g, trans2[1])) < threshold) for g in sg]):
            return 1
    else:
            return 0
def read_data(data_file):
    site_data = []
    trans_data = []
    dict_loader = yaml.load(data_file)
    vacancy_data = dict_loader.get('vacancy_data')  # vacancy data  
    site_data = dict_loader.get('site_data')  # site data
    trans_data = dict_loader.get('trans_data')  # transition data
    for i, data in enumerate(trans_data):
        if len(data) == 6:
            trans_data[i].append([0, 0.0, 0.0])
    bulk = dict_loader.get('bulk')      
    for i in range(len(site_data)):
        site_data[i][2]=0.0
        site_data[i][1]=1.0
    for i in range(len(trans_data)):
        trans_data[i][3]=0.0
        trans_data[i][5]=0.0
        if trans_data[i][6][0]:
            trans_data[i][6][1]=1e-6
            trans_data[i][6][2]=0.0
            trans_data[i][2]=1.0
            trans_data[i][4]=1.0
        else:
            trans_data[i][2]=0.5
            trans_data[i][4]=0.5
            
    w0_data = []
    w1_data = []
    w2_data = []
    for data in trans_data:
        if np.all(data[0]==[0,0,0]):
            w0_data.append(data)
        elif np.all(data[1]==[0,0,0]):
            w2_data.append(data)
        else:
            w1_data.append(data)
    return vacancy_data, site_data, w0_data, w1_data, w2_data        

In [2]:
## get data
data = open('Sn_data_temp.yaml', 'r')
vacancy_data, site_data, w0_data, w1_data, w2_data   = read_data(data)

In [3]:
## get list of deleted states and meta states
#for state in starset.states:
#    if state.i not in meta_sites and state.j in meta_sites:
#        if np.allclose(0.866025403784,np.linalg.norm(state.dx)):
#            to_del.append(state)
deleted_states = []
meta_states = []
new_w1_data = []
for jump in w1_data:
    if (jump[-1][0]):
        meta = np.array((np.array(jump[0])+np.array(jump[1]))/2).tolist()
        meta_states.append([meta,1e-6,0.0])
        new_jump1 = [jump[0],meta,jump[2],jump[3],jump[2],jump[3]]
        new_jump2 = [jump[1],meta,jump[4],jump[5],jump[4],jump[5]]
        new_w1_data.append(new_jump1)
        new_w1_data.append(new_jump2)
    else:
        deleted_states.append((np.array(jump[0])+np.array(jump[1]))/2)
        new_w1_data.append(jump)

In [4]:
deleted_states.append(np.array([0.5,0.5,0]))

In [5]:
a= 1.0#3.2342373809
c_a= np.sqrt(8/3) #1.5989108537
c=a*c_a
HCP = crystal.Crystal.HCP(a0=a,c_a=c_a, chemistry="Zr")
pg,sg = build_point_group_ops(HCP.lattice/a, HCP.basis[0])
len(pg)
print(HCP)
meta_basis = HCP.Wyckoffpos(np.array([5/6,2/3,0.25]))
basis = HCP.basis[0] + meta_basis
HCPmeta = crystal.Crystal(HCP.lattice, basis[0:8], chemistry=["Zr"], noreduce=True)
sitelist = HCPmeta.sitelist(0)
vacancyjumps = HCPmeta.jumpnetwork(0, 1.01*a)
meta_sites = np.arange(2,8,1)
for pos,jlist in enumerate(vacancyjumps):
        if np.any([np.allclose(dx,a*np.array([0.5, -0.8660254, 0.])) for (i,j), dx in jlist]):
            ind1 = pos
            break
#print("ind1 = ",ind1)
for pos,jlist in enumerate(vacancyjumps):
        if np.any([np.allclose(dx,a*np.array([ 0.25, -0.4330127, 0.])) for (i,j), dx in jlist]):
            ind2 = pos
            break
#print("ind2 = ",ind2)
jumpnetwork = [vacancyjumps[1], vacancyjumps[ind2]]
jumpnetwork2 = [vacancyjumps[1], vacancyjumps[ind1]]

#Lattice:
  a1 = [ 0.5       -0.8660254  0.       ]
  a2 = [ 0.5        0.8660254  0.       ]
  a3 = [ 0.          0.          1.63299316]
#Basis:
  (Zr) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Zr) 0.1 = [ 0.66666667  0.33333333  0.75      ]


In [6]:
print(HCPmeta)

#Lattice:
  a1 = [ 0.5       -0.8660254  0.       ]
  a2 = [ 0.5        0.8660254  0.       ]
  a3 = [ 0.          0.          1.63299316]
#Basis:
  (Zr) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Zr) 0.1 = [ 0.66666667  0.33333333  0.75      ]
  (Zr) 0.2 = [ 0.16666667  0.33333333  0.75      ]
  (Zr) 0.3 = [ 0.33333333  0.16666667  0.25      ]
  (Zr) 0.4 = [ 0.16666667  0.83333333  0.75      ]
  (Zr) 0.5 = [ 0.83333333  0.66666667  0.25      ]
  (Zr) 0.6 = [ 0.66666667  0.83333333  0.75      ]
  (Zr) 0.7 = [ 0.83333333  0.16666667  0.25      ]


In [68]:
starset = stars.StarSetMeta(jumpnetwork, HCPmeta, 0, meta_sites = meta_sites)
starset.generate(2)
to_del = []
for i, state in enumerate(starset.states):
    if state.i not in meta_sites and state.j in meta_sites:
        if state.i==0: 
            if np.any([apply_pg_site_dir(pg,np.dot(HCP.invlatt,state.dx), site) for site in deleted_states]):
                to_del.append(state)
                #print(i)
        elif state.i==1: 
            if np.any([apply_sg_site_dir(sg,np.dot(HCP.invlatt,state.dx), site) for site in deleted_states]):
                to_del.append(state)                
                #print(i)

In [69]:
HCPdiffuser = OnsagerCalc.VacancyMediatedMeta(HCPmeta, 0, sitelist, jumpnetwork, 2, meta_sites = np.arange(2,8,1), jumpnetwork2= jumpnetwork2, deleted_states=to_del)

In [70]:
binding_entropy_list = []
binding_energy_list = []
for i,state in enumerate(HCPdiffuser.interactlist()):
        data_not_found = 1
        #print(i,state)
        for site in site_data:
            if state.i==0: 
                if apply_pg_site_dir(pg,np.dot(HCP.invlatt,state.dx), site[0]):
                    #print(site[0],site[2])
                    binding_entropy_list.append(site[1])
                    binding_energy_list.append(site[2])
                    data_not_found = 0
                    break
            else:        
                if apply_sg_site_dir(sg,np.dot(HCP.invlatt,state.dx), site[0]):
                    #print(site[0],site[2])
                    binding_entropy_list.append(site[1])
                    binding_energy_list.append(site[2])
                    data_not_found = 0
                    break
        if data_not_found:
            for site in meta_states:
                if state.i==0: 
                    if apply_pg_site_dir(pg,np.dot(HCP.invlatt,state.dx), site[0]):
                        #print(site[0],site[2])
                        binding_entropy_list.append(site[1])
                        binding_energy_list.append(0.0)
                        data_not_found = 0
                        break
                else:        
                    if apply_sg_site_dir(sg,np.dot(HCP.invlatt,state.dx), site[0]):
                        #print(site[0],site[2])
                        binding_entropy_list.append(site[1])
                        binding_energy_list.append(0.0)
                        data_not_found = 0
                        break
        if data_not_found:
            if state.i in meta_sites or state.j in meta_sites:
                    #print("no data; setting binding energy = 0")
                    binding_entropy_list.append(1.0)
                    binding_energy_list.append(0.0)
            else:
                    #print("no data; setting binding energy = 0")
                    binding_entropy_list.append(1.0)
                    binding_energy_list.append(0.0)

In [80]:
##HCPtracer = {'preV': np.array([1.0,1.0/10000000.0]), 'eneV': np.array([0.0, 0.0]),
##             'preT0': np.array([ 1.0, 1.0]),
##             'eneT0': np.array([0, 0]),
##              }

# HCPtracer = {'preV': np.array([1.0,54.169024409/18.299152044],), 'eneV': np.array([0.0,0.51727]),
#              'preT0': np.array([ 54.169024409/9.26073917, 54.169024409/10.40702378]),
#              'eneT0': np.array([0.613339999999994,0.553549999999973]),
#               }

HCPsolute = {'preV': np.array([1.0,1e-6],), 'eneV': np.array([0.0,0.0]),
             'preT0': np.array([ 0.5, 1.0]),
             'eneT0': np.array([0.0,0.0]),
              }
HCPsolute['preSV'] = np.array(binding_entropy_list)
HCPsolute['eneSV'] = np.array(binding_energy_list)
HCPsolute['preS']= np.array([1.0,1.0])
HCPsolute['eneS']= np.array([0.,0])
# HCPtracer['preT2'] = np.array([ 54.169024409/9.26073917, 0.5* 54.169024409/10.40702378,54.169024409/10.40702378])
# HCPtracer['eneT2'] = np.array([0.613339999999994,0.553549999999973,1e12])


In [87]:
HCPsolute.update(HCPdiffuser.makeLIMBpreene(**HCPsolute))
for k,v in zip(HCPsolute.keys(), HCPsolute.values()): print(k,v)

eneT2 [ 0.  0.]
eneV [ 0.  0.]
preT2 [ 0.5  1. ]
eneT1 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]
preT0 [ 0.5  1. ]
preSV [ 1.  1.  1.  1.  1.  1.  1.  1.  1.]
preT1 [ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  1.   1.   1.   1.
  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1. ]
preV [  1.00000000e+00   1.00000000e-06]
eneT0 [ 0.  0.]
preS [ 1.  1.]
eneS [ 0.  0.]
eneSV [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [82]:
templist = []
for i,j in enumerate(HCPsolute['preT1']):
    if j==0.001:
        templist.append(i)
        HCPsolute['preT1'][i]=1.0

In [83]:
omega2=HCPdiffuser.omegalist(2)[0]
for j, (S1,S2) in enumerate(omega2):
    data_not_found = 1
    print(S1.i,S1.dx,S2.dx)
    if S1.i==0:
        for trans in w2_data:                        
            if apply_pg_site_dir(pg,np.dot(HCP.invlatt,S1.dx), trans[0]): 
                print(trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT2'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT2'][j]= trans[2]
                data_not_found = 0
                break
    else:
        for trans in w2_data:                        
            if apply_sg_site_dir(sg,np.dot(HCP.invlatt,S1.dx), trans[0]): 
                print(trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT2'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT2'][j]= trans[2]
                data_not_found = 0
                break
    if data_not_found:
            print("no data; limb used")
            #binding_entropy_list.append(1)
            #binding_energy_list.append(0.0)   



1 [-0.5        -0.28867513  0.81649658] [ 0.5         0.28867513 -0.81649658]
[0.33333333, 0.66666667, -0.5] [0.0, 0.0, 0.0] 0.5 0.0
0 [-0.5       -0.8660254  0.       ] [ 0.5        0.8660254  0.       ]
[1.0, 1.0, 0.0] [0.0, 0.0, 0.0] 0.5 0.0


In [84]:
omega1=HCPdiffuser.omegalist(1)[0]
for j, (S1,S2) in enumerate(omega1):
    data_not_found = 1    
    if S1.i==0:
        for i,trans in enumerate(new_w1_data):                        
            if apply_pg_trans(pg,np.array([np.dot(HCP.invlatt,S1.dx),np.dot(HCP.invlatt,S2.dx)]),np.array([trans[0],trans[1]])):
                #print(S1.i,S1.dx,S2.dx)
                print(j,i,trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT1'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT1'][j]= trans[2]
                data_not_found = 0
                break
            elif apply_pg_trans(pg,np.array([np.dot(HCP.invlatt,S1.dx),np.dot(HCP.invlatt,S2.dx)]),np.array([trans[1],trans[0]])):
                #print(S1.i,S1.dx,S2.dx)
                print(j,i,trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT1'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT1'][j]= trans[2]
                data_not_found = 0
                break
    elif S1.i==1:
        for i,trans in enumerate(new_w1_data):                              
            if apply_sg_trans(sg,np.array([np.dot(HCP.invlatt,S1.dx),np.dot(HCP.invlatt,S2.dx)]),np.array([trans[0],trans[1]])):
                #print(S1.i,S1.dx,S2.dx)
                print(j,i,trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT1'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT1'][j]= trans[2]
                data_not_found = 0
                break
            elif apply_sg_trans(sg,np.array([np.dot(HCP.invlatt,S1.dx),np.dot(HCP.invlatt,S2.dx)]),np.array([trans[1],trans[0]])):
                #print(S1.i,S1.dx,S2.dx)
                print(j,i,trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT1'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT1'][j]= trans[2]
                data_not_found = 0
                break
    if data_not_found:
            print("no data; limb used")
            #if np.isclose(HCPsolute['eneT1'][j],0.55355):
            #    HCPsolute['preT1'][j]=5.20504474229
            #print(HCPsolute['eneT1'][j],HCPsolute['preT1'][j])
            #binding_entropy_list.append(1)
            #binding_energy_list.append(0.0)      

0 2 [0.33333333, 0.66666667, -0.5] [0.0, 0.0, -1.0] 0.5 0.0
1 0 [0.33333333, 0.66666667, -0.5] [0.0, 1.0, -1.0] 0.5 0.0
2 1 [0.33333333, 0.66666667, -0.5] [0.0, 1.0, 0.0] 0.5 0.0
3 7 [1.0, 1.0, 0.0] [1.33333333, 1.66666667, -0.5] 0.5 0.0
4 6 [1.0, 1.0, 0.0] [1.33333333, 0.66666667, -0.5] 0.5 0.0
no data; limb used
no data; limb used
no data; limb used
no data; limb used
no data; limb used
no data; limb used
11 4 [0.33333333, 0.66666667, -0.5] [-0.66666667, 0.66666667, -0.5] 0.5 0.0
12 5 [0.33333333, 0.66666667, -0.5] [0.33333333, -0.33333333, -0.5] 0.5 0.0
13 11 [1.0, 1.0, 0.0] [1.0, 0.5, 0.0] 1.0 0.0
no data; limb used
no data; limb used
no data; limb used
17 3 [0.33333333, 0.66666667, -0.5] [0.33333333, 1.66666667, -0.5] 0.5 0.0
18 8 [1.0, 1.0, 0.0] [0.5, 1.0, 0.0] 1.0 0.0
no data; limb used
no data; limb used
no data; limb used
no data; limb used
no data; limb used
no data; limb used
no data; limb used


In [85]:
for k,v in zip(HCPsolute.keys(), HCPsolute.values()): print(k,v)

eneT2 [ 0.  0.]
eneV [ 0.  0.]
preT2 [ 0.5  0.5]
eneT1 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]
preT0 [ 0.5  1. ]
preSV [ 1.  1.  1.  1.  1.  1.  1.  1.  1.]
preT1 [ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  1.   1.
  1.   1.   0.5  1.   1.   1.   1.   1.   1.   1.   1. ]
preV [  1.00000000e+00   1.00000000e-06]
eneT0 [ 0.  0.]
preS [ 1.  1.]
eneS [ 0.  0.]
eneSV [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [86]:
#Temp=np.arange(500, 1010, 100)
#Temp = [500,1000]
Temp = [1000]
D_Onsager=[]
for T in Temp:
    pre=1e-8 # THz and Angstrom unit scaling
    Lvv, Lss, Lsv, L1vv = HCPdiffuser.Lij(*HCPdiffuser.preene2betafree(kB*T, **HCPsolute))
    #Lss=Lss*pre
    #D_Onsager.append([Lss[0,0],Lss[1,1],Lss[2,2]])        
    print(T, Lsv[0,0]/Lvv[0,0],Lsv[1,1]/Lvv[1,1],Lsv[2,2]/Lvv[2,2])
    print(T, Lsv[0,0]/Lss[0,0],Lsv[1,1]/Lss[1,1],Lsv[2,2]/Lss[2,2])
    Lss=Lss*pre
    Lvv=Lvv
    print(T, Lss[0,0],Lss[1,1],Lss[2,2])    
    print(T, Lvv[0,0],Lvv[1,1],Lvv[2,2])
    
D_Onsager=np.array(D_Onsager)

1000 -8.26174565923 -8.26174565923 -1.0
1000 -10.5756447926 -10.5756447926 -1.27967007336
1000 7.81202568366e-09 7.81202568366e-09 7.81449078808e-09
1000 0.999997000009 0.999997000009 0.999997000009


In [33]:
HCPdiffuser.vkinetic.vecvec[1]

[array([ -2.88675135e-01,   6.40987562e-17,   0.00000000e+00]),
 array([ 0.14433757, -0.25      ,  0.        ]),
 array([ 0.14433757,  0.25      ,  0.        ]),
 array([ -2.88675135e-01,  -6.40987562e-17,   0.00000000e+00]),
 array([ 0.14433757,  0.25      ,  0.        ]),
 array([-0.14433757,  0.25      ,  0.        ]),
 array([ 0.28867513,  0.        ,  0.        ]),
 array([-0.14433757, -0.25      ,  0.        ]),
 array([ 0.28867513,  0.        ,  0.        ]),
 array([ 0.14433757, -0.25      ,  0.        ]),
 array([-0.14433757,  0.25      ,  0.        ]),
 array([-0.14433757, -0.25      ,  0.        ])]

In [34]:
HCPdiffuser.vkinetic.vecvec[3]

[array([  2.88675135e-01,   6.40987562e-17,  -0.00000000e+00]),
 array([  2.88675135e-01,  -6.40987562e-17,  -0.00000000e+00]),
 array([-0.14433757,  0.25      , -0.        ]),
 array([-0.14433757, -0.25      , -0.        ]),
 array([-0.14433757, -0.25      , -0.        ]),
 array([ 0.14433757, -0.25      , -0.        ]),
 array([-0.28867513, -0.        , -0.        ]),
 array([ 0.14433757,  0.25      , -0.        ]),
 array([-0.28867513, -0.        , -0.        ]),
 array([-0.14433757,  0.25      , -0.        ]),
 array([ 0.14433757, -0.25      , -0.        ]),
 array([ 0.14433757,  0.25      , -0.        ])]

In [37]:
HCPdiffuser.vkinetic.vecvec[9]

[array([-0.        , -0.16666667,  0.23570226]),
 array([-0.14433757, -0.08333333, -0.23570226]),
 array([ 0.        ,  0.16666667, -0.23570226]),
 array([-0.14433757,  0.08333333,  0.23570226]),
 array([ 0.14433757, -0.08333333, -0.23570226]),
 array([ 0.14433757,  0.08333333,  0.23570226]),
 array([-0.14433757,  0.08333333, -0.23570226]),
 array([ 0.14433757, -0.08333333,  0.23570226]),
 array([-0.        , -0.16666667, -0.23570226]),
 array([ 0.14433757,  0.08333333, -0.23570226]),
 array([-0.14433757, -0.08333333,  0.23570226]),
 array([ 0.        ,  0.16666667,  0.23570226])]

In [35]:
HCPdiffuser.vkinetic.vecpos[1]

[8, 10, 12, 14, 18, 19, 20, 24, 25, 26, 30, 31]

In [36]:
HCPdiffuser.vkinetic.vecpos[3]

[9, 11, 13, 15, 16, 17, 21, 22, 23, 27, 28, 29]

In [38]:
HCPdiffuser.vkinetic.vecpos[9]

[56, 57, 58, 59, 61, 62, 64, 71, 74, 75, 78, 79]

In [40]:
HCPdiffuser.vkinetic.

[[0, 1, 2, 3, 4, 5],
 [6, 7],
 [8, 10, 12, 14, 18, 19, 20, 24, 25, 26, 30, 31],
 [9, 11, 13, 15, 16, 17, 21, 22, 23, 27, 28, 29],
 [32, 43, 46, 52, 54, 55],
 [33, 34, 35, 36, 40, 44, 45, 47, 49, 50, 51, 53],
 [37, 38, 39, 41, 42, 48],
 [56, 57, 58, 59, 61, 62, 64, 71, 74, 75, 78, 79],
 [60, 63, 65, 66, 67, 68, 69, 70, 72, 73, 76, 77],
 [80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103],
 [104,
  106,
  107,
  108,
  109,
  110,
  112,
  114,
  115,
  117,
  119,
  121,
  122,
  124,
  126,
  131,
  132,
  134,
  135,
  139,
  140,
  141,
  148,
  149],
 [105, 111, 118, 123, 125, 130, 133, 138, 142, 144, 146, 147],
 [113, 116, 120, 127, 128, 129, 136, 137, 143, 145, 150, 151],
 [152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163],
 [164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175],
 [176, 177, 178, 179],
 [180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
